In [1]:
# coding=utf-8
import pandas as pd
import csv  # Import csv
import tweepy
import re
import string
import nltk
import pandas as pd
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import numpy as np
pd.set_option('display.max_colwidth',1000)

In [2]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-

import codecs
#data = pd.read_csv('PreprocessedData.csv', header = None, encoding='utf-8', engine='python') ## for older one 
data = pd.read_csv('FinalListPreprocessedData.csv', header = None, encoding='utf-8', engine='python')
data.columns = ['ID','User','CREATED_AT','LABEL','stemming_words','LIKE COUNT','FOLLOWER COUNT']
data = data.iloc[1: , :] # start from first column
data = data.dropna()
data.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWER COUNT
1,2.0,1.42E+18,7/29/2021 2:40,Other,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොයා බැලීමේ වැඩ පිලිවලක් ඕනේ,1,1433
2,3.0,1.42E+18,7/27/2021 10:40,Politics,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා කරමින් හැකි ඉක්මණින් ආර්ථිකය,10,13898
3,4.0,1.42E+18,7/25/2021 20:56,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට යුරෝපීය රට වලට විසා නිකුත් නොකි,0,87
4,5.0,1.42E+18,7/25/2021 20:53,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට යුර,0,87
5,6.0,1.42E+18,7/25/2021 20:52,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට,1,87


In [3]:
data.loc[data['LABEL']=='Crime',"LABEL"] = 0
data.loc[data['LABEL']=='Entertainment',"LABEL"] = 1
data.loc[data['LABEL']=='Politics',"LABEL"] = 3
data.loc[data['LABEL']=='Sport',"LABEL"] = 4
data.loc[data['LABEL']=='Other',"LABEL"] = 5

In [4]:
data.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWER COUNT
1,2.0,1.42E+18,7/29/2021 2:40,5,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොයා බැලීමේ වැඩ පිලිවලක් ඕනේ,1,1433
2,3.0,1.42E+18,7/27/2021 10:40,3,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා කරමින් හැකි ඉක්මණින් ආර්ථිකය,10,13898
3,4.0,1.42E+18,7/25/2021 20:56,5,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට යුරෝපීය රට වලට විසා නිකුත් නොකි,0,87
4,5.0,1.42E+18,7/25/2021 20:53,5,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට යුර,0,87
5,6.0,1.42E+18,7/25/2021 20:52,5,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට,1,87


In [5]:
data['LABEL'].value_counts()

5    185
3    130
0    109
4     93
1     15
Name: LABEL, dtype: int64

In [6]:
############### Apply tfidf vectorizer ######################

In [7]:
data_x=data["stemming_words"]
data_y=data["LABEL"]

cv = TfidfVectorizer(analyzer = 'word', max_features = 10000, ngram_range=(1,3))

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=42)

In [9]:
x_traincv=cv.fit_transform(x_train)

In [10]:
a = x_traincv.toarray()
a

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.68112667, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
len(a[0])

9518

In [12]:
###########  MultiNomial Naive bayes ############

In [13]:
mnb = MultinomialNB()
y_train = y_train.astype('int')
mnb.fit(x_traincv,y_train)

MultinomialNB()

In [14]:
x_testcv=cv.transform(x_test)
prediction = mnb.predict(x_testcv)
prediction

array([5, 0, 4, 0, 5, 3, 0, 5, 3, 4, 0, 5, 5, 5, 4, 0, 5, 5, 3, 5, 5, 5,
       5, 5, 5, 5, 4, 5, 0, 5, 5, 5, 3, 5, 5, 0, 0, 3, 5, 5, 3, 4, 5, 5,
       5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 0, 5, 5, 3, 0, 5, 5, 0, 5, 5, 5,
       5, 5, 5, 3, 3, 4, 4, 5, 5, 5, 3, 5, 5, 3, 5, 5, 5, 3, 5, 5, 3, 5,
       3, 3, 3, 3, 5, 3, 4, 5, 5, 5, 0, 5, 4, 5, 3, 3, 0, 0, 5])

In [15]:
actual = np.array(y_test)
actual

array([5, 0, 4, 0, 5, 3, 0, 5, 5, 4, 0, 5, 4, 5, 1, 0, 5, 5, 0, 5, 3, 5,
       3, 4, 5, 5, 4, 5, 0, 5, 5, 5, 5, 5, 4, 0, 0, 3, 4, 4, 3, 4, 5, 4,
       3, 5, 0, 0, 3, 5, 3, 5, 5, 3, 4, 0, 4, 3, 3, 0, 5, 5, 0, 5, 4, 5,
       5, 4, 3, 3, 3, 1, 1, 4, 5, 5, 4, 5, 4, 3, 5, 5, 5, 5, 4, 5, 3, 5,
       3, 3, 0, 3, 5, 3, 4, 5, 4, 5, 0, 1, 4, 5, 3, 3, 0, 0, 5],
      dtype=object)

In [16]:
count = 0
for i in range (len(prediction)):
    if prediction[i] == actual[i]:
        count = count +1

In [17]:
count #correct prediction

74

In [18]:
len(prediction) #total prediction

107

In [19]:
67/107 ###### probbility

0.6261682242990654

In [20]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,prediction))

print(metrics.classification_report(y_test,prediction))


ValueError: Classification metrics can't handle a mix of unknown and multiclass targets